In [1]:
import pandas as pd
import pathlib
import os, os.path

In [2]:
dir_cur = pathlib.Path(os.getcwd())
input_data = dir_cur.joinpath("input_data")
output_data = dir_cur.joinpath("output_data")

In [3]:
gdp = pd.read_csv(input_data.joinpath("NY.GDP.MKTP.KD.csv")) 
gdp.head()

,year,NY.GDP.MKTP.KD
0,1982,4.983495e+09
1,1983,5.269775e+09
2,1984,5.251612e+09
3,1985,5.077973e+09
4,1986,5.097782e+09


In [4]:
gdp['gdp_mmm_usd'] = gdp['NY.GDP.MKTP.KD'] / 1000000000  # Convert to billions
gdp = gdp[gdp['year'] <= 2023].copy() # Filter out future years
gdp.tail()

,year,NY.GDP.MKTP.KD,gdp_mmm_usd
37,2019,3.960005e+10,39.600047
38,2020,4.076877e+10,40.768766
39,2021,4.221059e+10,42.210586
40,2022,4.414722e+10,44.147217
41,2023,4.650334e+10,46.503342


In [ ]:
# Define the growth rates for each period
growth_rates = [
    (2024, 2025, 8.35),   # 2021-2025 → usar para 2024 y 2025
    (2026, 2030, 8.22),   # 2026-2030
    (2031, 2035, 8.07),   # 2031-2035
    (2036, 2040, 7.83),   # 2036-2040
    (2041, 2050, 7.83)    # 2041-2050
]

In [6]:
last_gdp = gdp.iloc[-1]['gdp_mmm_usd']
projection = []

In [7]:
# Generate projections based on the defined growth rates
for start, end, rate in growth_rates:
    for year in range(start, end+1):
        last_gdp *= (1 + rate/100)
        projection.append({'year': year, 'gdp_mmm_usd': last_gdp})

# Combine the original GDP data with the projections        
gdp_extended = pd.concat([gdp, pd.DataFrame(projection)], ignore_index=True)        

In [8]:
# Display the first few rows of the extended GDP DataFrame
gdp_extended.head()

,year,NY.GDP.MKTP.KD,gdp_mmm_usd
0,1982,4.983495e+09,4.983495
1,1983,5.269775e+09,5.269775
2,1984,5.251612e+09,5.251612
3,1985,5.077973e+09,5.077973
4,1986,5.097782e+09,5.097782


In [9]:
# Display the last few rows of the extended GDP DataFrame
gdp_extended.tail()

,year,NY.GDP.MKTP.KD,gdp_mmm_usd
64,2046,NaN,273.741762
65,2047,NaN,295.175742
66,2048,NaN,318.288002
67,2049,NaN,343.209953
68,2050,NaN,370.083292


In [10]:
# Prepare the final DataFrame with the required columns
gdp_extended['Year'] = gdp_extended['year']
gdp_extended['iso_code3'] = 'UGA'
gdp_extended['Nation'] = 'Uganda'

gdp_extended = gdp_extended[['iso_code3', 'Year', 'gdp_mmm_usd', 'Nation']]  
gdp_extended.head()

,iso_code3,Year,gdp_mmm_usd,Nation
0,UGA,1982,4.983495,Uganda
1,UGA,1983,5.269775,Uganda
2,UGA,1984,5.251612,Uganda
3,UGA,1985,5.077973,Uganda
4,UGA,1986,5.097782,Uganda


In [11]:
gdp_extended.to_csv(output_data.joinpath("gdp_mmm_usd.csv"), index=False)